In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
import tensorflow as tf
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Rescaling, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import KLDivergence

In [ ]:
import efficientnet.tfkeras as efn
from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.optimizers import Adam
from keras.losses import KLDivergence
from keras.callbacks import EarlyStopping

# Build model
rn_model = Sequential()

# Base EfficientNet model
pretrained_model = efn.EfficientNetB0(
    include_top=False,
    weights=None,
    input_shape=(400, 256, 3)  # Adjust input shape as needed
)

In [ ]:
import efficientnet.tfkeras as efn

inp = tf.keras.Input(shape=(400,256,3))
base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=None)

# OUTPUT
x = base_model(inp)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)

# COMPILE MODEL
model = tf.keras.Model(inputs=inp, outputs=x)
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
loss = tf.keras.losses.KLDivergence()

model.compile(loss=loss, optimizer = opt)

In [ ]:
efn_model = Sequential()

pretrained_model = tf.keras.applications.efficientnet.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(400, 256, 3),
    pooling='average',
    classes=6,
    classifier_activation='softmax',
)

for layer in pretrained_model.layers[:-1]:
  layer.trainable = False

efn_model.add(pretrained_model)
efn_model.add(Flatten())
efn_model.add(Dense(512, activation='relu'))
efn_model.add(Dense(6, activation='softmax'))

loss = KLDivergence()
opt = Adam(learning_rate=0.000000001)
callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
efn_model.compile(optimizer=opt, loss=loss)

In [ ]:
efn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional  (None, 13, 8, 1280)       4049571   
 )                                                               
                                                                 
 flatten_4 (Flatten)         (None, 133120)            0         
                                                                 
 dense_9 (Dense)             (None, 512)               68157952  
                                                                 
 dense_10 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 72210601 (275.46 MB)
Trainable params: 68161030 (260.01 MB)
Non-trainable params: 4049571 (15.45 MB)
_________________________________________________________________


In [ ]:
rn_model = Sequential()

pretrained_model = keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(400, 256, 3),
    pooling='average',
    classes=6,
    classifier_activation="softmax",
)

for layer in pretrained_model.layers:
  layer.trainable = False

rn_model.add(pretrained_model)
rn_model.add(Flatten())
rn_model.add(Dense(512, activation='relu'))
rn_model.add(Dense(6, activation='softmax'))

rn_model.summary()

In [ ]:
batch_size=16
num_batches = 23  # Calculate total number of batches
for i in range(1, num_batches + 1):
    batch_path = f'/content/drive/MyDrive/kaggle/X_data_batch_{i*743}.npy'
    X_batch = np.load(batch_path)

    # Split X_batch into smaller batches of size 16
    num_sub_batches = len(X_batch) // batch_size
    for j in range(num_sub_batches):
        start_idx = j * batch_size
        end_idx = (j + 1) * batch_size
        X_sub_batch = X_batch[start_idx:end_idx]

        # Feed X_sub_batch into your model for training or evaluation
        rn_model.fit(X_sub_batch, y_data[(i-1)*len(X_batch) + start_idx:(i-1)*len(X_batch) + end_idx], epochs=1, validation_split=0.2)

1/1 [==============================] - 1s 1s/step - loss: nan - val_loss: nan


KeyboardInterrupt: 

In [ ]:
import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

rn_model = Sequential()

pretrained_model = keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_shape=(400, 256, 3),
    pooling='average',
    classes=6,
    classifier_activation="softmax",
)

for layer in pretrained_model.layers[:-2]:
  layer.trainable = False

rn_model.add(pretrained_model)
rn_model.add(Flatten())
rn_model.add(Dense(512, activation='relu'))
rn_model.add(Dense(6, activation='softmax'))

rn_model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 13, 8, 2048)       23564800  
                                                                 
 flatten_8 (Flatten)         (None, 212992)            0         
                                                                 
 dense_17 (Dense)            (None, 512)               109052416 
                                                                 
 dense_18 (Dense)            (None, 6)                 3078      
                                                                 
Total params: 132620294 (505.91 MB)
Trainable params: 109059590 (416.03 MB)
Non-trainable params: 23560704 (89.88 MB)
_________________________________________________________________


In [ ]:
loss = KLDivergence()
opt = Adam(learning_rate=0.000000001)
callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
rn_model.compile(optimizer=opt, loss=loss)

In [ ]:
batch_size=16
num_batches = 23  # Calculate total number of batches
for i in range(1, num_batches + 1):
    batch_path = f'/content/drive/MyDrive/kaggle/X_data_batch_{i*743}.npy'
    X_batch = np.load(batch_path)

    # Split X_batch into smaller batches of size 16
    num_sub_batches = len(X_batch) // batch_size
    for j in range(num_sub_batches):
        start_idx = j * batch_size
        end_idx = (j + 1) * batch_size
        X_sub_batch = X_batch[start_idx:end_idx]

        # Feed X_sub_batch into your model for training or evaluation
        rn_model.fit(X_sub_batch, y_data[(i-1)*len(X_batch) + start_idx:(i-1)*len(X_batch) + end_idx], epochs=1, validation_split=0.2)  # Example for training

1/1 [==============================] - 2s 2s/step - loss: 1.9493 - val_loss: 4.4336


KeyboardInterrupt: 

In [ ]:
batch_size=16
# Split X_batch into smaller batches of size 16
num_sub_batches = len(X_data) // batch_size
for j in range(num_sub_batches):
    start_idx = j * batch_size
    end_idx = (j + 1) * batch_size
    X_sub_batch = X_data[start_idx:end_idx]

    # Feed X_sub_batch into your model for training
    rn_model.fit(X_sub_batch, y_data[start_idx:end_idx], epochs=1)

1/1 [==============================] - 2s 2s/step - loss: 2.8675
